# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import pickle

In [2]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [4]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fb9409b9a58>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fb9409ff978>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, num_epochs=0)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |



NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fb9409b9a58>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fb9409ff978>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.53859999999999997

In [11]:
net.fit(X, y, num_epochs=20)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       1 |      0.26446 | 0.04354 |
|       2 |      0.21478 | 0.03548 |
|       3 |      0.20260 | 0.03568 |
|       4 |      0.19586 | 0.03729 |
|       5 |      0.19142 | 0.03786 |
|       6 |      0.18807 | 0.03821 |
|       7 |      0.18542 | 0.03920 |
|       8 |      0.18319 | 0.04124 |
|       9 |      0.18112 | 0.04344 |
|      10 |      0.17941 | 0.03942 |
|      11 |      0.17766 | 0.04208 |
|      12 |      0.17608 | 0.03982 |
|      13 |      0.17447 | 0.03942 |
|      14 |      0.17284 | 0.03625 |
|      15 |      0.17140 | 0.03844 |
|      16 |      0.16991 | 0.03589 |
|      17 |      0.16849 | 0.03602 |
|      18 |      0.16719 | 0.03647 |
|      19 |      0.16571 | 0.03616 |
|      20 |      0.16

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fb9409b9a58>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fb9409ff978>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [12]:
y_proba = net.predict_proba(X)

In [13]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.94240000000000002

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [14]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [15]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [16]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [17]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [18]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] hidden__num_units=50, update__learning_rate=0.1, max_epochs=5 ...
[CV]  hidden__num_units=50, update__learning_rate=0.1, max_epochs=5, score=0.899820 -   0.3s
[CV] hidden__num_units=50, update__learning_rate=0.1, max_epochs=5 ...
[CV]  hidden__num_units=50, update__learning_rate=0.1, max_epochs=5, score=0.908818 -   0.3s
[CV] hidden__num_units=50, update__learning_rate=0.1, max_epochs=5 ...
[CV]  hidden__num_units=50, update__learning_rate=0.1, max_epochs=5, score=0.905762 -   0.4s
[CV] hidden__num_units=50, update__learning_rate=0.5, max_epochs=5 ...
[CV]  hidden__num_units=50, update__learning_rate=0.5, max_epochs=5, score=0.910018 -   0.4s
[CV] hidden__num_units=50, update__learning_rate=0.5, max_epochs=5 ...
[CV]  hidden__num_units=50, update__learning_rate=0.5, max_epochs=5, score=0.911818 -   0.5s
[CV] hidden__num_units=50, update__learning_rate=0.5, max_epochs=5 ...
[CV]  hidden__num_units=50, update__learning_rate

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   18.8s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=Fa...ject at 0x7fb9409ff978>,),
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'hidden__num_units': [50, 100], 'update__learning_rate': [0.1, 0.5], 'max_epochs': [5, 10]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [19]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 100, 'max_epochs': 10, 'update__learning_rate': 0.5},
 0.91459999999999997)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [20]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [21]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [22]:
net = NeuralNetRegressor(l2, verbose=1)

In [23]:
net.set_params(update__learning_rate=0.0001)

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fb9409b9cf8>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fb9409b9cc0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

### Train

In [24]:
net.fit(X, y, num_epochs=20)

# Neural Network with 20401 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |    100 |
|   1 | dense  |    200 |
|   2 | dense  |      1 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       1 |  43436.25781 | 0.06574 |
|       2 |  41256.64844 | 0.02810 |
|       3 |  30908.43750 | 0.02936 |
|       4 |  10005.61914 | 0.02930 |
|       5 |   1217.02173 | 0.02852 |
|       6 |    496.41934 | 0.02957 |
|       7 |    427.42319 | 0.02958 |
|       8 |    392.87958 | 0.02943 |
|       9 |    365.78607 | 0.03087 |
|      10 |    342.27985 | 0.03342 |
|      11 |    321.13290 | 0.03185 |
|      12 |    301.70123 | 0.03196 |
|      13 |    283.62457 | 0.03222 |
|      14 |    266.73914 | 0.03432 |
|      15 |    250.89449 | 0.03242 |
|      16 |    235.99590 | 0.03241 |
|      17 |    221.93008 | 0.02947 |
|      18 |    208.67709 | 0.02913 |
|      19 |    196.21332 | 0.03022 |
|      20 |    184.4

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fb9409b9cf8>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fb9409b9cc0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

## Saving and restoring

### Save previous net

In [25]:
score_before = mean_squared_error(y, net.predict(X))
print(score_before)

177.47590574


In [26]:
with open('mynet.pkl', 'wb') as f:
    pickle.dump(net, f)

### Create a new net with same architecture

In [27]:
with open('mynet.pkl', 'rb') as f:
    new_net = pickle.load(f)

In [28]:
score_after = mean_squared_error(y, new_net.predict(X))
print(score_after)
assert np.isclose(score_before, score_after)

177.47590574
